In [1]:
using CLOUD.ConservationLaws
using CLOUD.SpatialDiscretizations
using CLOUD.Solvers
using CLOUD.InitialConditions
using CLOUD.Mesh
using CLOUD.IO
using OrdinaryDiffEq
using TimerOutputs

"Set up paths"
plots_path = "../plots/advection1d_demo/"
results_path = "../results/advection1d_demo/"

"Define the physical problem"
a = 1.0  # advection velocity
A = 1.0  # amplitude
L = 1.0  # domain length
k = 2*π/L  # wave number
T = 1*L/a  # end time

# constant-coefficient linear adection equation with sine wave initial condition
initial_data = InitialDataSine(A,k)
conservation_law = linear_advection_equation(a)
u_exact = initial_condition(initial_data, conservation_law)

"Set discretization parameters"
M = 100  # number of elements
p = 2  # degree of discretization
N_q = 3  # number of quadrature point
reference_approximation=ReferenceApproximation(DGSEM(p), 
    Line(), LGQuadrature())
form = StrongConservationForm()
dt_scale=0.1

"Generate a uniform periodic mesh"
mesh = uniform_periodic_mesh(reference_approximation.reference_element, 
    (0.0,L), M)

"Construct spatial discretization"
spatial_discretization = SpatialDiscretization(mesh, reference_approximation)
    
ode_problem = semidiscretize(conservation_law, 
    spatial_discretization,
    initial_data, form,
    (0.0, T), Eager());

┌ Info: Precompiling CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2]
└ @ Base loading.jl:1342


LoadError: MethodError: no method matching /(::LinearMaps.UniformScalingMap{Bool}, ::LinearAlgebra.Diagonal{Float64, Vector{Float64}})
[0mClosest candidates are:
[0m  /(::Any, [91m::ChainRulesCore.AbstractThunk[39m) at /Users/tristanmontoya/.julia/packages/ChainRulesCore/bxKCw/src/tangent_types/thunks.jl:33
[0m  /([91m::Union{LinearAlgebra.AbstractTriangular, StridedMatrix{T} where T}[39m, ::LinearAlgebra.Diagonal) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/diagonal.jl:492
[0m  /([91m::LinearAlgebra.Adjoint{T, var"#s814"} where {T, var"#s814"<:(AbstractVector{T} where T)}[39m, ::AbstractMatrix{T} where T) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/adjtrans.jl:325
[0m  ...

In [ ]:
"run the solver"
sol = solve(ode_problem, RK4(), adaptive=false, 
    dt=dt_scale*(L/M)/(a*(2*p+1)), save_everystep=false)
print_timer()
reset_timer!();

In [ ]:
"visualize the solution"
plotter = Plotter(spatial_discretization, plots_path)
visualize(last(sol.u), plotter,
    "final.pdf", exact_solution=u_exact,
     label="U^h(x,t)", label_exact="U(x,t)")